# **Install the requird packages**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


# **Import required packages**

In [ ]:
from openai import AzureOpenAI

import pandas as pd
import numpy as np
import json
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, RocCurveDisplay
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Configs

/content
/content/drive/MyDrive/Colab Notebooks/Configs
/content/drive/MyDrive/Colab Notebooks/Configs
total 1
-rw------- 1 root root 110 Apr 27 02:57 configs.json


In [ ]:
with open(r'configs.json') as config_file:
    config_details = json.load(config_file)

# [link text](https:// [link text](https://))**Create Azure 'gpt-35-turbo' and 'text-embedding-ada-002' clients**

In [ ]:
api_key = config_details['AZURE-OPEN-API-KEY']
api_base = config_details['ENDPOINT']
api_version = "2024-02-01"
deployment_name = "gpt-35-turbo"
gpt_model_name = "gpt-35-turbo"

client = AzureOpenAI(
 api_key=api_key,
 api_version=api_version,
 base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

In [ ]:
api_key = config_details['AZURE-OPEN-API-KEY']
api_base = config_details['ENDPOINT']
api_version = "2024-02-01"
deployment_name = "text-embedding-ada-002"
gpt_model_name = "text-embedding-ada-002"

client_embd = AzureOpenAI(
 api_key=api_key,
 api_version=api_version,
 base_url=f"{api_base}/openai/deployments/{deployment_name}"
)
# POST https://{your-resource-name}.openai.azure.com/openai/deployments/{deployment-id}/embeddings?api-version={api-version}

In [ ]:
csv_file_path = '/content/drive/MyDrive/SEPSIS_DataSet'
%cd /content/drive/MyDrive/SEPSIS_DataSet
!ls

/content/drive/MyDrive/SEPSIS_DataSet
Sepsis_Analysis.sql			  SEPSIS_POS_CHART_EVENTS.csv
SEPSIS_NEG_AGE.csv			  sepsis_positive_pivoted.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
SEPSIS_NEG_CHART_EVENTS.csv		  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  test_data.csv
sepsis_neg_pivoted.csv			  train_data.csv
SEPSIS_POS_AGE.csv			  validate_data.csv
SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv


**Iteration 1:**
<B>Verify the model performance with 8 features as per SEPSIS 3 definition.

8 features are:
Invasive Blood Pressure systolic,
Invasive Blood Pressure mean,
Temperature (Fahrenheit),
Respiratory Rate (RR),
White Blood Cells (cells/mm^3),
Heart Rate.

There is a 30% noise data.

Sepsis postive patients: 624

Sepsis negative patients: 624
</B>

In [ ]:
###   LOAD DATA  POSTIVE AND NEGATIVE SEPSIS DATASET ###
positive  = pd.read_csv(csv_file_path+'/sepsis_positive_pivoted.csv')
negative  = pd.read_csv(csv_file_path+'/sepsis_neg_pivoted.csv')

In [ ]:
print(positive.subject_id.count())
print(negative.subject_id.count())

624
16839


In [ ]:
postive = positive[positive.age < 120]
negative = negative[negative.age < 120]

In [ ]:
print(positive.subject_id.count())
print(negative.subject_id.count())

624
15268


In [ ]:
# positive.groupby('Arterial Blood Pressure diastolic').size()
# positive.groupby('Arterial Blood Pressure diastolic')['subject_id'].value_counts().unstack(fill_value=0)
# positive.groupby('Arterial Blood Pressure systolic')['subject_id'].value_counts().unstack(fill_value=0)
positive.isna().sum()

subject_id                                 0
age                                        0
Arterial Blood Pressure diastolic        620
Arterial Blood Pressure systolic         620
Heart Rate                                 3
Invasive Blood Pressure diastolic        613
Invasive Blood Pressure mean             264
Invasive Blood Pressure systolic         264
Non-Invasive Blood Pressure diastolic    426
Non-Invasive Blood Pressure systolic     613
Respiratory Rate (RR)                      3
Temperature (Fahrenheit)                 261
WBC Count (cells/uL)                     621
White Blood Cells (cells/mm^3)             1
dtype: int64

In [ ]:
negative.isna().sum()

subject_id                                   0
age                                          0
Arterial Blood Pressure diastolic        15043
Arterial Blood Pressure systolic         15043
Heart Rate                                  68
Invasive Blood Pressure diastolic        15064
Invasive Blood Pressure mean              6730
Invasive Blood Pressure systolic          6757
Non-Invasive Blood Pressure diastolic     9268
Non-Invasive Blood Pressure systolic     15047
Respiratory Rate (RR)                       80
Temperature (Fahrenheit)                  6483
White Blood Cells (cells/mm^3)              58
dtype: int64

In [ ]:
positive.head()

,subject_id,age,Arterial Blood Pressure diastolic,Arterial Blood Pressure systolic,Heart Rate,Invasive Blood Pressure diastolic,Invasive Blood Pressure mean,Invasive Blood Pressure systolic,Non-Invasive Blood Pressure diastolic,Non-Invasive Blood Pressure systolic,Respiratory Rate (RR),Temperature (Fahrenheit),WBC Count (cells/uL),White Blood Cells (cells/mm^3),OUTPUT_LABEL
0,21,88,NaN,NaN,79.0 BPM,NaN,60.0 mmHg,114.0 mmHg,43.0 mmHg,NaN,12.0 BPM,99.7 Deg. F,NaN,30.5 K/uL,1
1,94,74,NaN,NaN,77.0 BPM,NaN,66.0 mmHg,124.0 mmHg,NaN,NaN,13.0 BPM,99.8 Deg. F,NaN,4.6 K/uL,1
2,157,81,NaN,NaN,115.0 BPM,NaN,82.0 mmHg,109.0 mmHg,59.0 mmHg,NaN,30.0 BPM,98.4 Deg. F,NaN,52.0 K/uL,1
3,166,300,NaN,NaN,132.0 BPM,NaN,54.0 mmHg,109.0 mmHg,NaN,NaN,38.0 BPM,102.2 Deg. F,NaN,16.5 K/uL,1
4,450,80,NaN,NaN,107.0 BPM,NaN,60.0 mmHg,118.0 mmHg,NaN,NaN,25.0 BPM,99.8 Deg. F,NaN,16.2 K/uL,1


In [ ]:
negative.head()

,subject_id,age,Arterial Blood Pressure diastolic,Arterial Blood Pressure systolic,Heart Rate,Invasive Blood Pressure diastolic,Invasive Blood Pressure mean,Invasive Blood Pressure systolic,Non-Invasive Blood Pressure diastolic,Non-Invasive Blood Pressure systolic,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3),OUTPUT_LABEL
0,6,66,NaN,NaN,87.0 BPM,NaN,40.0 mmHg,138.0 mmHg,56.0 mmHg,NaN,13.0 BPM,96.8 Deg. F,10.6 K/uL,0
1,12,72,NaN,NaN,86.0 BPM,NaN,72.0 mmHg,144.0 mmHg,68.0 mmHg,NaN,12.0 BPM,95.9 Deg. F,7.7 K/uL,0
3,20,76,NaN,NaN,80.0 BPM,NaN,54.0 mmHg,138.0 mmHg,59.0 mmHg,NaN,12.0 BPM,96.8 Deg. C,17.5 K/uL,0
4,22,65,NaN,NaN,85.0 BPM,NaN,65.0 mmHg,130.0 mmHg,NaN,NaN,18.0 BPM,97.6 Deg. F,5.1 K/uL,0
5,23,71,NaN,NaN,90.0 BPM,NaN,60.0 mmHg,106.0 mmHg,62.0 mmHg,NaN,7.0 BPM,95.0 Deg. C,7.6 K/uL,0


**Labelling the data and sample the sepsis negative dataset to match the positive sepsis data set.**

In [ ]:
#label the data set
positive['OUTPUT_LABEL'] = 1
negative['OUTPUT_LABEL'] = 0

In [ ]:
df_sepsis_pos = positive.iloc[:,[1, 4, 6, 7, 8, 10, 11, 13, 14]]
df_sepsis_neg = negative.iloc[:,[1, 4, 6, 7, 8, 10, 11, 12, 13]]

In [ ]:
df_sepsis_neg_sample = df_sepsis_neg.sample(n = len(df_sepsis_pos), random_state = 42)
df_sepsis_neg_sample

,age,Heart Rate,Invasive Blood Pressure mean,Invasive Blood Pressure systolic,Non-Invasive Blood Pressure diastolic,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3),OUTPUT_LABEL
8995,70,100.0 BPM,60.0 mmHg,120.0 mmHg,46.0 mmHg,14.0 BPM,96.44 Deg. C,8.13 K/uL,0
4970,82,138.0 BPM,38.0 mmHg,73.0 mmHg,67.0 mmHg,25.0 BPM,98.7 Deg. F,15.1 K/uL,0
3283,76,52.0 BPM,52.0 mmHg,106.0 mmHg,69.0 mmHg,18.0 BPM,95.1 Deg. F,8.2 K/uL,0
11341,71,91.0 bpm,NaN,NaN,NaN,12.0 insp/min,NaN,8.6 K/uL,0
7869,77,92.0 BPM,67.0 mmHg,134.0 mmHg,NaN,27.0 BPM,95.1 Deg. F,5.9 K/uL,0
...,...,...,...,...,...,...,...,...,...
6421,66,84.0 BPM,44.0 mmHg,97.0 mmHg,58.0 mmHg,10.0 BPM,99.68 Deg. C,4.7 K/uL,0
16603,67,79.0 bpm,NaN,NaN,NaN,12.0 insp/min,NaN,12.0 K/uL,0
4148,77,60.0 BPM,58.0 mmHg,119.0 mmHg,79.0 mmHg,13.0 BPM,96.9 Deg. F,8.7 K/uL,0
12774,65,91.0 bpm,NaN,NaN,NaN,12.0 insp/min,NaN,7.3 K/uL,0


In [ ]:
df_sepsis_pos

,age,Heart Rate,Invasive Blood Pressure mean,Invasive Blood Pressure systolic,Non-Invasive Blood Pressure diastolic,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3),OUTPUT_LABEL
0,88,79.0 BPM,60.0 mmHg,114.0 mmHg,43.0 mmHg,12.0 BPM,99.7 Deg. F,30.5 K/uL,1
1,74,77.0 BPM,66.0 mmHg,124.0 mmHg,NaN,13.0 BPM,99.8 Deg. F,4.6 K/uL,1
2,81,115.0 BPM,82.0 mmHg,109.0 mmHg,59.0 mmHg,30.0 BPM,98.4 Deg. F,52.0 K/uL,1
3,300,132.0 BPM,54.0 mmHg,109.0 mmHg,NaN,38.0 BPM,102.2 Deg. F,16.5 K/uL,1
4,80,107.0 BPM,60.0 mmHg,118.0 mmHg,NaN,25.0 BPM,99.8 Deg. F,16.2 K/uL,1
...,...,...,...,...,...,...,...,...,...
619,88,78.0 bpm,NaN,NaN,NaN,15.0 insp/min,NaN,13.8 K/uL,1
620,70,151.0 bpm,NaN,NaN,NaN,31.0 insp/min,NaN,8.5 K/uL,1
621,76,135.0 bpm,NaN,NaN,NaN,34.0 insp/min,NaN,10.6 K/uL,1
622,84,86.0 bpm,NaN,NaN,NaN,29.0 insp/min,NaN,11.1 K/uL,1


Positive and Negative Samples and Data Splitting

In [ ]:
df = pd.concat([df_sepsis_pos,df_sepsis_neg_sample], ignore_index=True)

In [ ]:
print('Number of positive samples:', (df.OUTPUT_LABEL == 1).sum())
print('Number of negative samples:',  (df.OUTPUT_LABEL == 0).sum())
print('Total samples:', len(df))

Number of positive samples: 624
Number of negative samples: 624
Total samples: 1248


**Split the dataset. Taining data set - 70% and Test dataset - 30%**

In [ ]:
# shuffle the samples
df = df.sample(n = len(df), random_state = 42)
df = df.reset_index(drop = True)

# Save 30% of the data as validation and test data
df_valid_test=df.sample(frac=0.30,random_state=42)

df_test = df_valid_test.sample(frac = 0.5, random_state = 42)
df_valid = df_valid_test.drop(df_test.index)

# use the rest of the data as training data
df_train_all=df.drop(df_valid_test.index)

print('Test prevalence(n = %d):'%len(df_test),df_test.OUTPUT_LABEL.sum()/ len(df_test))
print('Valid prevalence(n = %d):'%len(df_valid),df_valid.OUTPUT_LABEL.sum()/ len(df_valid))
print('Train all prevalence(n = %d):'%len(df_train_all), df_train_all.OUTPUT_LABEL.sum()/ len(df_train_all))
print('all samples (n = %d)'%len(df))
assert len(df) == (len(df_test)+len(df_valid)+len(df_train_all)),'math didnt work'

Test prevalence(n = 187): 0.5133689839572193
Valid prevalence(n = 187): 0.5508021390374331
Train all prevalence(n = 874): 0.4862700228832952
all samples (n = 1248)


In [ ]:
# split the training data into positive and negative
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

# merge the balanced data
df_train = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 42)],axis = 0)

# shuffle the order of training samples
df_train = df_train.sample(n = len(df_train), random_state = 42).reset_index(drop = True)

print('Train prevalence (n = %d):'%len(df_train), df_train.OUTPUT_LABEL.sum()/ len(df_train))

Train prevalence (n = 850): 0.5


In [ ]:
df_train

,age,Heart Rate,Invasive Blood Pressure mean,Invasive Blood Pressure systolic,Non-Invasive Blood Pressure diastolic,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3),OUTPUT_LABEL
0,79,101.0 BPM,74.0 mmHg,139.0 mmHg,NaN,16.0 BPM,100.1 Deg. F,13.2 K/uL,0
1,71,100.0 BPM,70.0 mmHg,88.0 mmHg,NaN,17.0 BPM,101.1 Deg. F,0.2 K/uL,1
2,66,93.0 bpm,NaN,NaN,NaN,13.0 insp/min,NaN,26.5 K/uL,1
3,68,122.0 BPM,72.0 mmHg,139.0 mmHg,61.0 mmHg,23.0 BPM,95.4 Deg. F,5.6 K/uL,0
4,73,70.0 BPM,96.0 mmHg,141.0 mmHg,37.0 mmHg,28.0 BPM,97.5 Deg. F,17.5 K/uL,1
...,...,...,...,...,...,...,...,...,...
845,68,138.0 bpm,NaN,NaN,NaN,23.0 insp/min,NaN,35.0 K/uL,1
846,78,87.0 BPM,49.0 mmHg,110.0 mmHg,NaN,30.0 BPM,100.8 Deg. F,12.6 K/uL,1
847,84,82.0 BPM,54.0 mmHg,105.0 mmHg,85.0 mmHg,20.0 BPM,98.3 Deg. F,9.4 K/uL,1
848,78,117.0 BPM,44.0 mmHg,102.0 mmHg,NaN,15.0 BPM,96.3 Deg. F,16.3 K/uL,0


**Functions for generating the prompts and embeddings**

In [ ]:
# Create column information to pass to GPT
class ConditionsDataFrame(Dataset):
  def __init__(self, df):
      self.df = df

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      column_names = [
          ("age", "Age: "),
          ("Heart Rate", " - Heart Rate: "),
          ("Invasive Blood Pressure mean", " - Invasive Blood Pressure mean: "),
          ("Invasive Blood Pressure systolic", " - Invasive Blood Pressure systolic: "),
          ("Non-Invasive Blood Pressure diastolic", " - Non-Invasive Blood Pressure diastolic: "),
          ("Respiratory Rate (RR)", " - Respiratory Rate (RR): "),
          ("Temperature (Fahrenheit)", " - Temperature (Fahrenheit): "),
          ("White Blood Cells (cells/mm^3)", " - White Blood Cells (cells/mm^3): ")
      ]

      x_strs = [f"{col_desc}{self.df.iloc[index][col]}" for col, col_desc in column_names]
      x_str = ''.join(x_strs)
      x_str = x_str.replace('\n', '')
      # x_str = 'Decide in a single word if this patient has SEPSIS. ' +x_str
      x_str = x_str+ '. Decide in a single word if this patient has SEPSIS? Yes or no?'

      return x_str

def generate_embeddings(texts, model="text-embedding-ada-002"):
    embeddings = []
    for text in tqdm(texts):
        text = text.replace("\n", " ")
        response = client_embd.embeddings.create(input = [text], model=model).data[0].embedding
        embeddings.append(response)
    return np.array(embeddings)

**Run the GPT embedding model**

In [ ]:
embeddings_train = generate_embeddings(prompts_train)

100%|██████████| 850/850 [00:57<00:00, 14.72it/s]


In [ ]:
np.shape(embeddings_train)

(850, 1536)

In [ ]:
labels_train = list(df_train['OUTPUT_LABEL'])

**Train a Logistic Regression classifier on the embeddings to predict the positive or negative SEPSIS**

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(embeddings_train, labels_train)

LogisticRegression(max_iter=1000)

**LR model prediction on Test Dataset**

In [ ]:
prompts_test = ConditionsDataFrame(df_test) #  test data embeddings
embeddings_test = generate_embeddings(prompts_test)

100%|██████████| 187/187 [00:12<00:00, 14.55it/s]


In [ ]:
labels_test = list(df_test['OUTPUT_LABEL'])

In [ ]:
test_pred = model.predict_proba(embeddings_test)[:,1]
auroc = roc_auc_score(labels_test, test_pred)
auprc = average_precision_score(labels_test, test_pred)
print('\nAUROC:', auroc, '\nAUPRC', auprc)


AUROC: 0.6624313186813188 
AUPRC 0.7059050122070958


**LR model prediction on Validate Dataset**

In [ ]:
prompts_valid = ConditionsDataFrame(df_valid) #  valid set data embeddings
embeddings_valid = generate_embeddings(prompts_valid)

100%|██████████| 187/187 [00:11<00:00, 15.67it/s]


In [ ]:
labels_valid = list(df_valid['OUTPUT_LABEL'])

In [ ]:
valid_pred = model.predict_proba(embeddings_valid)[:,1]
auroc = roc_auc_score(labels_valid, valid_pred)
auprc = average_precision_score(labels_valid, valid_pred)
print('\nAUROC:', auroc, '\nAUPRC', auprc)


AUROC: 0.6978733240869165 
AUPRC 0.7732409329177167


**Iteration 2:**
<B>Check the model performance as per SEPSIS 2 definition. total 4 features. 0% noise data.

Sepsis train datas set: 361

Sepsis test and validate data set: 155
</B>

In [ ]:
!ls -lart

total 2071996
-rw------- 1 root root       9701 Apr 25 15:03 SEPSIS_POS_AGE.csv
-rw------- 1 root root     397260 Apr 25 15:05 SEPSIS_NEG_AGE.csv
-rw------- 1 root root   38418867 Apr 25 15:47 SEPSIS_POS_CHART_EVENTS.csv
-rw------- 1 root root 1366766848 Apr 25 15:51 SEPSIS_NEG_CHART_EVENTS.csv
-rw------- 1 root root   21852853 Apr 25 16:22 SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
-rw------- 1 root root  670536369 Apr 25 16:25 SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv
-rw------- 1 root root     634285 Apr 25 16:32 SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
-rw------- 1 root root     633948 Apr 25 16:36 SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
-rw------- 1 root root   21256488 Apr 25 16:37 SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv
-rw------- 1 root root      12240 Apr 25 17:30 Sepsis_Analysis.sql
-rw------- 1 root root      43142 Apr 26 03:48 sepsis_positive_pivoted.csv
-rw------- 1 root root    1133518 Apr 26 22:31 sepsis_neg_pivoted.csv
-rw------- 1 root root       3672 Apr 28 06:

In [ ]:
###   LOAD DATA  POSTIVE AND NEGATIVE SEPSIS ###
df_train_clean  = pd.read_csv(csv_file_path+'/train_data.csv')
df_test_clean  = pd.read_csv(csv_file_path+'/test_data.csv')
df_validate_clean  = pd.read_csv(csv_file_path+'/validate_data.csv')

**Verify the size and columns details**

In [ ]:
print(np.shape(df_train_clean))
print(np.shape(df_test_clean))
print(np.shape(df_validate_clean))

(361, 6)
(78, 6)
(77, 6)


In [ ]:
df_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      361 non-null    int64 
 1   Heart Rate                      361 non-null    object
 2   Respiratory Rate (RR)           361 non-null    object
 3   Temperature (Fahrenheit)        361 non-null    object
 4   White Blood Cells (cells/mm^3)  361 non-null    object
 5   target                          361 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.0+ KB


**Define the Prompt function**

In [ ]:
LLM_PROMPT = '. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer.'

In [ ]:
class patientData(object):
  'This class represents the patient data. It allows us to track for sepsis or not'
  def __init__(self, medicalData = '', sepsisBoolean = ''):
    self.medicalData = medicalData
    self.sepsisBoolean = sepsisBoolean

# Create column information to pass to GPT
class ConditionsDataFrame(Dataset):
  def __init__(self, df):
      self.df = df
      self.patient = patientData()

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      column_names = [
          ("Heart Rate", "The patient's heart rate is "),
          ("Respiratory Rate (RR)", ", respiratory rate is "),
          ("Temperature (Fahrenheit)", ", temperature is "),
          ("White Blood Cells (cells/mm^3)", ", white blood cell count is ")
      ]

      x_strs = [f"{col_desc}{self.df.iloc[index][col]}" for col, col_desc in column_names]
      x_str = ''.join(x_strs)
      x_str = x_str.replace('\n', '')
      x_str = x_str+ LLM_PROMPT

      return x_str

**Training, Test and Validate Data prompts**

In [ ]:
prompts_train_clean = ConditionsDataFrame(df_train_clean) #  training data prompts
prompts_test_clean = ConditionsDataFrame(df_test_clean) #  test data prompts
prompts_validate_clean = ConditionsDataFrame(df_validate_clean) #  validate data prompts

**Verify the prompts**

In [ ]:
print(prompts_train_clean[0])
print(prompts_test_clean[0])
print(prompts_validate_clean[0])

The patient's heart rate is 94.0 BPM, respiratory rate is 24.0 BPM, temperature is 95.2 Deg. F, white blood cell count is 14.7 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer.
The patient's heart rate is 80.0 BPM, respiratory rate is 22.0 BPM, temperature is 97.6 Deg. F, white blood cell count is 9.7 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer.
The patient's heart rate is 77.0 BPM, respiratory rate is 18.0 BPM, temperature is 96.9 Deg. F, white blood cell count is 13.3 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer.


**Generate Embeddings**

In [ ]:
embeddings_train_clean = generate_embeddings(prompts_train_clean)
embeddings_test_clean = generate_embeddings(prompts_test_clean)
embeddings_validate_clean = generate_embeddings(prompts_validate_clean)

100%|██████████| 77/77 [00:07<00:00, 10.29it/s]


In [ ]:
labels_train_clean = list(df_train_clean['target'])
labels_test_clean = list(df_test_clean['target'])
labels_validate_clean = list(df_validate_clean['target'])

**Train a simple logistic regression classifier on embeddings**

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(embeddings_train_clean, labels_train_clean)

LogisticRegression(max_iter=1000)

**Predictions on Test Data and print model performance metrics**

In [ ]:
test_pred = model.predict_proba(embeddings_test_clean)[:,1]

In [ ]:
# Get list of ground truth targets
targets = list(labels_test_clean)
boolean_targets = [s.strip('.').lower() == 'yes' for s in targets]
test_labels = [1 if x else 0 for x in boolean_targets]

In [ ]:
auroc = roc_auc_score(test_labels, test_pred)
auprc = average_precision_score(test_labels, test_pred)

print('\nAUROC:', auroc, '\nAUPRC:', auprc)


AUROC: 0.7440476190476191 
AUPRC: 0.7592067698925661


**Predictions on Validate Data and print model performance metrics**

In [ ]:
validate_pred = model.predict_proba(embeddings_validate_clean)[:,1]

In [ ]:
# Get list of ground truth targets
targets = list(labels_validate_clean)
boolean_targets = [s.strip('.').lower() == 'yes' for s in targets]
validate_labels = [1 if x else 0 for x in boolean_targets]

In [ ]:
auroc = roc_auc_score(validate_labels, validate_pred)
auprc = average_precision_score(validate_labels, validate_pred)

print('\nAUROC:', auroc, '\nAUPRC:', auprc)


AUROC: 0.7213225371120108 
AUPRC: 0.7314391637924489


**Quick selection of best model**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
# Get list of ground truth targets of training dataset
targets = list(labels_train_clean)
boolean_targets = [s.strip('.').lower() == 'yes' for s in targets]
train_labels = [1 if x else 0 for x in boolean_targets]

In [ ]:
#pipelines of models( it is short was to fit and pred)
pipeline_lr=Pipeline([('lr_classifier',LogisticRegression(random_state=42))])

pipeline_dt=Pipeline([ ('dt_classifier',DecisionTreeClassifier(random_state=42))])

pipeline_rf=Pipeline([('rf_classifier',RandomForestClassifier())])

pipeline_svc=Pipeline([('sv_classifier',SVC())])

pipeline_xgb=Pipeline([('xg_classifier',XGBClassifier())])

# List of all the pipelines
pipelines = [pipeline_lr, pipeline_dt, pipeline_rf, pipeline_svc, pipeline_xgb]

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'RandomForest', 3: "SVC", 4: "XGBoost"}


# Fit the pipelines
for pipe in pipelines:
    pipe.fit(embeddings_train_clean, train_labels)

#cross validation on accuracy
cv_results_accuracy = []
for i, model in enumerate(pipelines):
    cv_score = cross_val_score(model, embeddings_train_clean,train_labels, cv=10 )
    cv_results_accuracy.append(cv_score)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

Logistic Regression: 0.659309 
Decision Tree: 0.639790 
RandomForest: 0.717492 
SVC: 0.506907 
XGBoost: 0.709159 


In [ ]:
#Building a dictionary with list of optional values that will me analyesed by GridSearch CV
parameters = {
    'n_estimators': [100,150, 200,500,700,900],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,12,14,16],
    'criterion' :['gini', 'entropy'],
    'n_jobs':[-1,1,None]
}

In [ ]:
#Fitting the trainingset to find parameters with best accuracy

model_rfc_params = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameters, cv= 5)
model_rfc_params.fit(embeddings_train_clean, train_labels)

#Getting the optimal parameters to use in RandomForest

model_rfc_params.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

{'criterion': 'entropy',
 'max_depth': 14,
 'max_features': 'log2',
 'n_estimators': 150,
 'n_jobs': -1}

**Select the RandomForest best parameters**

In [ ]:
{'criterion': 'entropy',
 'max_depth': 14,
 'max_features': 'log2',
 'n_estimators': 150,
 'n_jobs': -1}

In [ ]:
# Ramdom Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(**model_rfc_params.best_params_)
model.fit(embeddings_train_clean, train_labels)

RandomForestClassifier(criterion='entropy', max_depth=14, max_features='log2',
                       n_estimators=150, n_jobs=-1)

**Predictions on Test Data and print model performance metrics**

In [ ]:
test_pred = model.predict_proba(embeddings_test_clean)[:,1]

# Get list of ground truth targets
targets = list(labels_test_clean)
boolean_targets = [s.strip('.').lower() == 'yes' for s in targets]
test_labels = [1 if x else 0 for x in boolean_targets]

auroc = roc_auc_score(test_labels, test_pred)
auprc = average_precision_score(test_labels, test_pred)

print('\nAUROC:', auroc, '\nAUPRC:', auprc)


AUROC: 0.7093253968253969 
AUPRC: 0.7134563935017051


**Predictions on Validate Data and print model performance metrics**

In [ ]:
validate_pred = model.predict_proba(embeddings_validate_clean)[:,1]

# Get list of ground truth targets
targets = list(labels_validate_clean)
boolean_targets = [s.strip('.').lower() == 'yes' for s in targets]
validate_labels = [1 if x else 0 for x in boolean_targets]

auroc = roc_auc_score(validate_labels, validate_pred)
auprc = average_precision_score(validate_labels, validate_pred)

print('\nAUROC:', auroc, '\nAUPRC:', auprc)


AUROC: 0.7361673414304993 
AUPRC: 0.7197129203840932
